#Creating environment and Downloading Data

In [3]:
!pip install kaggle

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"saptarshimandal30","key":"0fc968fc729d983dd70056ddec3fe9c8"}'}

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

##Downloading Data

In [6]:
!kaggle competitions download -c accelerometer-biometric-competition

  0% 0.00/692k [00:00<?, ?B/s]
100% 692k/692k [00:00<00:00, 47.6MB/s]
  0% 0.00/574k [00:00<?, ?B/s]
100% 574k/574k [00:00<00:00, 72.5MB/s]
 96% 239M/249M [00:03<00:00, 79.4MB/s]
100% 249M/249M [00:04<00:00, 63.3MB/s]
100% 238M/239M [00:03<00:00, 83.3MB/s]
100% 239M/239M [00:03<00:00, 74.0MB/s]


In [7]:
!unzip "/content/train.csv.zip"

Archive:  /content/train.csv.zip
  inflating: train.csv               


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
import scipy as sc

#Data Preparation

##Data

In [9]:
df = pd.read_csv("/content/train.csv.zip")
df.head()

,T,X,Y,Z,Device
0,1.336645e+12,0.340509,8.308413,4.140585,7
1,1.336645e+12,0.381370,8.390134,4.249548,7
2,1.336645e+12,0.272407,8.471856,4.018002,7
3,1.336645e+12,0.149824,8.430995,4.290409,7
4,1.336645e+12,0.272407,8.430995,4.481094,7


In [10]:
labels = df.iloc[:,4].values
unique_labels=np.unique(labels)

In [11]:
length = 15000

In [12]:

data_size=[]
for i in range(100):
  a = unique_labels[i]
  j=0
  while(labels[j]!=a):
    j+=1
  size=0
  while(labels[j]==a):
    size+=1
    j+=1
  data_size.append(size)
low_sizes=[]
for i in range(100):
  if data_size[i]<=length:
    low_sizes.append(i)
low_sizes

In [13]:
unique_labels=np.delete(unique_labels,low_sizes) # deleting user datas for less available data

##Distributing User data to an ARRAY

In [14]:
N_Users=10
k=0
n=unique_labels[0]           # All m_id in a list
X=[0 for j in range(N_Users)] # Final Array
for m_id in unique_labels[:N_Users]:
    n=unique_labels[0]
    i=0                   # index keeping variable
    while(n!=m_id):      # Loop for searching the index of first m_id datapoint
        n=df.iloc[i,4]  # Label of i-th index datapoint
        i+=1
    temp=[]
    n=m_id
    while(n==m_id):     # Loop For inserting all datapoints of same m_id
        x_y_z = [df.iloc[i,1],df.iloc[i,2],df.iloc[i,3]]
        temp.append(x_y_z)
        n=df.iloc[i,4]
        i+=1
    X[k]=temp
    k+=1 #incrementing [Final Array] indices

##Algo for creating windows

In [15]:
# Main Algo for Creating Windows .....#1

def create(df,wind_size,hop_size,m_index):
  #l=((len(df.x)-wind_size)/hop_size)+1
  rows, cols = (wind_size, 3) 
  #a = [[0 for i in range(cols)] for j in range(rows)] 
  array= [0 for i in range(int((length-wind_size)/hop_size)+1)]
  k = 0
  for i in range(0,(length-wind_size+1),hop_size):
      a=create1(df,i,wind_size,m_index)
       #array.append(a)
      array[k] = a
      k+=1
  #return pd.DataFrame(array)
  return array


In [16]:
# Main Algo for Creating Windows .....#2

def create1(df,x,w,m_index):
     rows = w 
     cols = 12
     
     arr = [[0 for i in range(cols)] for j in range(rows)] 
     for i in range(0,w,1):
        arr[i][0]=df[m_index][x+i][0]
        arr[i][1]=df[m_index][x+i][1]
        arr[i][2]=df[m_index][x+i][2]
     arr = np.array(arr)
     for i in range(0,w,1):
        arr[i][3]=np.mean(arr,axis=0)[0]
        arr[i][4]=np.mean(arr,axis=0)[1]
        arr[i][5]=np.mean(arr,axis=0)[2]
        arr[i][6]=np.std(arr,axis=0)[0]
        arr[i][7]=np.std(arr,axis=0)[1]
        arr[i][8]=np.std(arr,axis=0)[2]
        arr[i][9]=abs(sc.fft(arr[:,0]))[i]
        arr[i][10]=abs(sc.fft(arr[:,1]))[i]
        arr[i][11]=abs(sc.fft(arr[:,2]))[i]
     return arr.flatten()

##Final Array for training

In [17]:
N_Users

10

In [18]:
X_final = [0 for i in range( N_Users)]
for m_index in range(N_Users):
  X_final[m_index]=create(X,225,10,m_index)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Using scipy.fft as a function is deprecated and will be removed in SciPy 1.5.0, use scipy.fft.fft instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Using scipy.fft as a function is deprecated and will be removed in SciPy 1.5.0, use scipy.fft.fft instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Using scipy.fft as a function is deprecated and will be removed in SciPy 1.5.0, use scipy.fft.fft instead.


In [19]:
X_input=[]
for i in range(N_Users):
  temp = X_final[i]
  X_input += temp
X_input = np.array(X_input)

### Creating Labels

In [20]:
y = [0 for i in range(X_input.shape[0])]
for i in range(X_input.shape[0]):
  y[i]=int(i*N_Users/X_input.shape[0])
y = np.array(y)

## Splitting into Training and Testing

In [21]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
from sklearn import svm
from sklearn import metrics
X_train , X_test , y_train , y_test = train_test_split(X_input,y,test_size = 0.2,shuffle=True)

In [22]:
X_train.shape

(11824, 2700)

# BASE LINES

##ML classifiers

In [23]:
clf_Logistic_Reg = sklearn.linear_model.LogisticRegression()
clf_SVM = sklearn.svm.SVC()
clf_dt = sklearn.tree.DecisionTreeClassifier()
clf_rf = sklearn.ensemble.RandomForestClassifier()
clf_knn = sklearn.neighbors.KNeighborsClassifier()

In [24]:

import time
start_time = time.time()
clf_Logistic_Reg.fit(X_train,y_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 23.77745509147644 seconds ---


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
import time
start_time = time.time()
clf_SVM.fit(X_train,y_train)
print("--- %s seconds ---" % (time.time() - start_time))

NameError: ignored

In [ ]:

start_time = time.time()
clf_dt.fit(X_train,y_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 266.4858658313751 seconds ---


In [ ]:


start_time = time.time()
clf_rf.fit(X_train,y_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 377.5618872642517 seconds ---


In [ ]:

start_time = time.time()
clf_knn.fit(X_train,y_train)
print("--- %s seconds ---" % (time.time() - start_time))

##Metrics

###Logistic Regression

In [25]:
y_pred_train_Log = clf_Logistic_Reg.predict(X_train)
y_pred_test_Log = clf_Logistic_Reg.predict(X_test)

In [26]:
acc_train = metrics.accuracy_score(y_train,y_pred_train_Log)
acc_test = metrics.accuracy_score(y_test,y_pred_test_Log)
print('Logistic Regression:')
print('Accuracy: %.4f' % acc_train,'val_acc: %.4f' %acc_test)

Logistic Regression:
Accuracy: 0.6278 val_acc: 0.6049


In [27]:
recall = metrics.recall_score(y_test, y_pred_test_Log, labels=[1,2], average='micro')
precision = metrics.precision_score(y_test, y_pred_test_Log, labels=[1,2], average='micro')
f1 = metrics.f1_score(y_test, y_pred_test_Log, labels=[1,2], average='micro')
print('Recall: %.4f' % recall,'Precision: %.4f' % precision,'F-Measure: %.4f' % f1)

Recall: 0.4784 Precision: 0.5383 F-Measure: 0.5066


###SVM

In [ ]:
y_pred_train_svm = clf_SVM.predict(X_train)
y_pred_test_svm = clf_SVM.predict(X_test)

In [ ]:
y_pred_train_svm.shape

(41384,)

In [ ]:
acc_train = metrics.accuracy_score(y_train,y_pred_train_svm)
acc_test = metrics.accuracy_score(y_test,y_pred_test_svm)
print('SVM:')
print('Accuracy: %.4f' % acc_train, 'val_acc: %.4f' %acc_test)

SVM:
Accuracy: 0.6641 val_acc: 0.6509


In [ ]:
recall = metrics.recall_score(y_test, y_pred_test_svm, labels=[1,2], average='micro')
precision = metrics.precision_score(y_test,y_pred_test_svm, labels=[1,2], average='micro')
f1 = metrics.f1_score(y_test, y_pred_test_svm, labels=[1,2], average='micro')
print('Recall: %.4f' % recall,'Precision: %.4f' % precision,'F-Measure: %.4f' % f1)

Recall: 0.6225 Precision: 0.3933 F-Measure: 0.4821


###Decision Tree

In [ ]:
y_pred_train_dt = clf_dt.predict(X_train)
y_pred_test_dt = clf_dt.predict(X_test)

In [ ]:
acc_train = metrics.accuracy_score(y_train,y_pred_train_dt)
acc_test = metrics.accuracy_score(y_test,y_pred_test_dt)
print('Decision Tree:')
print('Accuracy: %.4f' % acc_train,'val_acc: %.4f' %acc_test)

Decision Tree:
Accuracy: 1.0000 val_acc: 0.7298


In [ ]:
recall = metrics.recall_score(y_test, y_pred_test_dt, labels=[1,2], average='micro')
precision = metrics.precision_score(y_test,y_pred_test_dt, labels=[1,2], average='micro')
f1 = metrics.f1_score(y_test, y_pred_test_dt, labels=[1,2], average='micro')
print('Recall: %.4f' % recall,'Precision: %.4f' % precision,'F-Measure: %.4f' % f1)

Recall: 0.6417 Precision: 0.6417 F-Measure: 0.6417


###Random Forest

In [ ]:
y_pred_train_rf = clf_rf.predict(X_train)
y_pred_test_rf = clf_rf.predict(X_test)

In [ ]:
acc_train = metrics.accuracy_score(y_train,y_pred_train_rf)
acc_test = metrics.accuracy_score(y_test,y_pred_test_rf)
print('Random Forest:')
print('Accuracy: %.4f' % acc_train,'val_acc: %.4f' %acc_test)

Random Forest:
Accuracy: 1.0000 val_acc: 0.9178


In [ ]:
recall = metrics.recall_score(y_test, y_pred_test_rf, labels=[1,2], average='micro')
precision = metrics.precision_score(y_test,y_pred_test_rf, labels=[1,2], average='micro')
f1 = metrics.f1_score(y_test, y_pred_test_rf, labels=[1,2], average='micro')
print('Recall: %.4f' % recall,'Precision: %.4f' % precision,'F-Measure: %.4f' % f1)

Recall: 0.8939 Precision: 0.8224 F-Measure: 0.8567


###KNN

In [ ]:
y_pred_train_knn = clf_knn.predict(X_train)
y_pred_test_knn = clf_knn.predict(X_test)

In [ ]:
acc_train = metrics.accuracy_score(y_train,y_pred_train_knn)
acc_test = metrics.accuracy_score(y_test,y_pred_test_knn)
print('KNN:')
print('Accuracy: %.4f' % acc_train,'val_acc: %.4f' %acc_test)

In [ ]:
recall = metrics.recall_score(y_test, y_pred_test_knn, labels=[1,2], average='micro')
precision = metrics.precision_score(y_test,y_pred_test_knn, labels=[1,2], average='micro')
f1 = metrics.f1_score(y_test,y_pred_test_knn, labels=[1,2], average='micro')
print('Recall: %.4f' % recall,'Precision: %.4f' % precision,'F-Measure: %.4f' % f1)

In [ ]:
import sklearn.metrics
import seaborn as sns
cf_matrix = sklearn.metrics.confusion_matrix(y_test,y_pred_test_dt)
sns.heatmap(cf_matrix)